In [ ]:
import requests
import json
import os
import re
import shutil
import subprocess
import sys
import time
import yt_dlp
import zipfile
from tkinter import *
from tkinter import ttk
import tkinter as tk
from tkinter import filedialog
from tkinter import font
from tkinter import messagebox
from tkinter import Listbox
import os
import json
import shutil
import os
import re
import subprocess
import time
from seleniumwire import webdriver
#pip install blinker==1.4 in case the import webdriver doesn't work
from selenium.webdriver.firefox.options import Options

def getTitle(url):
  try:
    print(url)
    
    # Get the video title
    result = subprocess.run([
        sys.executable, "-m", "yt_dlp",
        "--no-playlist",
        "--skip-download",
        "--get-title",
        url
    ], capture_output=True, text=True, check=True)
    
    # Extract the title from the result
    title = result.stdout.strip()
    return title
  except subprocess.CalledProcessError as e:
    return ""


def filter_details(file, url, episode_file):

  # Get correct format options
  with open(file, 'r') as f:
    data = json.load(f)

    # Get info for details.json file
    transformed_data = {
        "title": data.get("title", ""),
        "author": data.get("uploader", ""),
        "artist": data.get("uploader", "can't re-use uploader I guess..."),
        "description": data.get("description", ""),
        "genre": data.get("tags", []),
        # Trying 3 for the status value to see if artist/uploader works
        "status": "2",
        "url": url,
    }

    # Get details for episode.json file
    episode_data = [{
              "episode_number": 1,
              "name": data.get("title", ""),
              "date_upload": data.get("upload_date", "")
    }]

    url_pattern = r'https?://\S+|www\.\S+'

    # Replace URLs with an empty string
    transformed_data["description"] = re.sub(url_pattern, '', transformed_data["description"])

    # Rewrite .json files WITH PROPER SYNTAX!!!
    with open(file, 'w') as f:
      json.dump(transformed_data, f, indent=4)

    with open(episode_file, 'w') as f:
      json.dump(episode_data, f, indent=4)
  return

















In [4]:
def download(url):
    try:
        # Get the video title
        title = getTitle(url)
        
        if title:
            # Make path to new directory including video's title
            path = os.path.join(os.getcwd(), title)
            
            print(f"Path of stored title: {path}")
            print(f"Title: {title}")
            ffmpeg_dir = os.path.join(os.getcwd(), "ffmpeg")
            subprocess.run([
                sys.executable, "-m", "yt_dlp",
                '--ffmpeg-location', ffmpeg_dir,
                '--convert-thumbnails', 'jpg',
                '--progress',
                '--no-playlist',
                '--write-thumbnail',
                '--embed-thumbnail',
                '--add-metadata',
                '--write-info-json',
                '--remux-video', 'mkv',
                '-f', 'bestvideo[height<=1080]+bestaudio/best[height<=1080]/best',
                '-o', f'{path}\\%(title)s.%(ext)s',
                url
            ])
            
            # TODO: Rename cover image to "cover.jpg"
            if os.path.exists(f'{path}/{title}.jpg'):
              os.rename(f'{path}/{title}.jpg', f'{path}/cover.jpg')
            else:
              print("No cover image found.")
            
            # Create empty .nomedia file
            os.chdir(path)
            with open('.nomedia', 'w') as makefile:
              pass
            os.chdir('..')
            
            # Change info.json file to be "details.json"
            if os.path.exists(f'{path}/{title}.info.json'):
              os.rename(f'{path}/{title}.info.json', f'{path}/details.json')
              filter_details(f'{path}/details.json', url, f'{path}/episode.json')
              return True
            else:
              print("No info file found.")
              return title
        
    except subprocess.CalledProcessError as e:
        print("………………………………………………………………………………………………………")
        print(f"An error occurred: {e}")
        print(f"Failed to download url: {url}")
        print("………………………………………………………………………………………………………")
        with open('failed_downloads.txt', 'a') as f:
          f.write(url + '\n')
        return False

In [ ]:
url = ""


In [ ]:
def instantiate_checkpoint_files():
  with open('urls.txt', 'w') as f:
    f.write(input("Enter urls: "))
    pass
  with open('completed.txt', 'w') as f:
    pass
  with open('failed_downloads.txt', 'w') as f:
    pass
  with open('downloaded already.txt', 'w') as f:
    pass

In [ ]:
# Check if the infofiles have been instantiated
if not os.path.exists('urls.txt'):
  instantiate_checkpoint_files()

# Read Checkpoint files
with open('completed.txt', 'r') as f:
  completed = f.read().splitlines()[:]
with open('failed_downloads.txt', 'r') as f:
  failed = f.read().splitlines()[:]
with open('downloaded already.txt', 'r') as f:
  zipped = f.read().splitlines()[:]
  
title = getTitle(url)
if url not in completed and url not in failed and title + '.zip' not in zipped:
  if download(url):
    title = getTitle(url)
    with open('completed.txt', 'a') as f:
      f.write(url + '\n')
else:
  print('SKIPPED: ' + title + '.zip')